In [1]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')

node = np.empty(4, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.PS)
node[2] = Source(model,'Source')
node[3] = Sink(model,'Sink')

jobclass = np.empty(2, dtype=object)
jobclass[0] = ClosedClass(model, 'ClosedClass', 2, node[0], 0)
jobclass[1] = OpenClass(model, 'OpenClass', 0)

node[0].set_service(jobclass[0], Erlang(3,2))
node[0].set_service(jobclass[1], HyperExp(0.5,3,10))

node[1].set_service(jobclass[0], HyperExp(0.1,1,10))
node[1].set_service(jobclass[1], Exp(1))

node[2].set_arrival(jobclass[1], Exp(0.1))

M = model.get_number_of_stations()
K = model.get_number_of_classes()

P = model.init_routing_matrix()

# Closed class routing: Delay <-> Queue1 (circul(2))
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)
P.set(jobclass[0], jobclass[0], node[1], node[0], 1.0)

# Open class routing: Source -> Queue1 -> Sink
P.set(jobclass[1], jobclass[1], node[2], node[1], 1.0)
P.set(jobclass[1], jobclass[1], node[1], node[3], 1.0)
            
model.link(P)

In [3]:
print('This example shows the execution of the solver on a 2-class 2-node mixed model.')

# Configure solver options
options = Solver.default_options()
options['keep'] = True
options['verbose'] = 1
options['cutoff'] = 3
options['seed'] = 23000

# Create multiple solvers for comparison
solver = np.array([], dtype=object)
solver = np.append(solver, CTMC(model, cutoff=3))  # CTMC is infinite on this model
solver = np.append(solver, JMT(model, seed=23000, verbose=True, keep=False))
solver = np.append(solver, SSA(model, seed=23000, verbose=False, samples=10000))
solver = np.append(solver, FLD(model))
solver = np.append(solver, MVA(model))
solver = np.append(solver, NC(model))
solver = np.append(solver, MAM(model))
solver = np.append(solver, DES(model, seed=23000))

# Solve and display results
AvgTable = np.empty(len(solver), dtype=object)
for s in range(len(solver)):
    try:
        print(f'\nSOLVER: {solver[s].get_name()}')
        AvgTable[s] = solver[s].avg_table()
        print(AvgTable[s])
    except Exception as e:
        print(f'Error with {solver[s].get_name()}: {e}')

This example shows the execution of the solver on a 2-class 2-node mixed model.

SOLVER: CTMC
CTMC solver using state space cutoff = 3 for open/mixed model.
Station    JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay ClosedClass 1.4362 1.4362 0.6667  0.6667 2.1542 2.1542
 Queue1 ClosedClass 0.5638 0.4093 0.2617  0.2617 2.1542 2.1542
 Queue1   OpenClass 0.1724 0.0998 1.7280  1.7239 0.0998 0.0998
 Source ClosedClass 0.0000 0.0000 0.0000  0.0000 1.0000 1.0000
 Source   OpenClass 0.0000 0.0000 0.0000  0.0000 0.1000 0.1000
  Station     JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay  ClosedClass  1.4362  1.4362  0.6667  0.6667  2.1542  2.1542
1  Queue1  ClosedClass  0.5638  0.4093  0.2617  0.2617  2.1542  2.1542
2  Queue1    OpenClass  0.1724  0.0998  1.7280  1.7239  0.0998  0.0998
3  Source  ClosedClass  0.0000  0.0000  0.0000  0.0000  1.0000  1.0000
4  Source    OpenClass  0.0000  0.0000  0.0000  0.0000  0.1000  0.1000

SOLVER: JMT
SolverJMT command: java -c

/home/gcasale/Dropbox/code/line-dev.git/python/line_solver/api/solvers/ctmc/handler.py:834: UserWarning: State space truncation may cause inaccurate results. Consider varying cutoff to assess sensitivity.
  warnings.warn(


Station    JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay ClosedClass 1.4243 1.4243 0.6641  0.6641 2.1692 2.1692
  Delay   OpenClass 0.0000 0.0000 0.0000  0.0000 0.0000 0.0000
 Queue1 ClosedClass 0.5757 0.4114 0.2688  0.2688 2.1687 2.1687
 Queue1   OpenClass 0.1739 0.1106 1.7335  1.7335 0.1015 0.1015
 Source ClosedClass 0.0000 0.0000 0.0000  0.0000 0.0000 1.0000
 Source   OpenClass 0.0000 0.0000 0.0000  0.0000 0.0000 0.1000
  Station     JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay  ClosedClass  1.4243  1.4243  0.6641  0.6641  2.1692  2.1692
1   Delay    OpenClass  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
2  Queue1  ClosedClass  0.5757  0.4114  0.2688  0.2688  2.1687  2.1687
3  Queue1    OpenClass  0.1739  0.1106  1.7335  1.7335  0.1015  0.1015
4  Source  ClosedClass  0.0000  0.0000  0.0000  0.0000  0.0000  1.0000
5  Source    OpenClass  0.0000  0.0000  0.0000  0.0000  0.0000  0.1000

SOLVER: SSA


Station    JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay ClosedClass 1.4171 1.4171 0.6576  0.6576 2.1549 2.1549
 Queue1 ClosedClass 0.5829 0.4065 0.2705  0.2705 2.1549 2.1549
 Queue1   OpenClass 0.2334 0.1284 2.1298  2.1298 0.1096 0.1096
 Source ClosedClass 0.0000 0.0000 0.0000  0.0000 0.0000 1.0000
 Source   OpenClass 0.0000 0.0000 0.0000  0.0000 0.0000 0.1000
  Station     JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay  ClosedClass  1.4171  1.4171  0.6576  0.6576  2.1549  2.1549
1  Queue1  ClosedClass  0.5829  0.4065  0.2705  0.2705  2.1549  2.1549
2  Queue1    OpenClass  0.2334  0.1284  2.1298  2.1298  0.1096  0.1096
3  Source  ClosedClass  0.0000  0.0000  0.0000  0.0000  0.0000  1.0000
4  Source    OpenClass  0.0000  0.0000  0.0000  0.0000  0.0000  0.1000

SOLVER: Fluid
Station    JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay ClosedClass 1.5565 1.5565 0.6667  0.6667 2.3348 2.3348
 Queue1 ClosedClass 0.4435 0.4435 0.1900  0.1900 2.3340

/home/gcasale/Dropbox/code/line-dev.git/python/line_solver/api/solvers/mva/amvald.py:271: RuntimeWarning: invalid value encountered in divide
  deltaclass = (Nchain - 1) / Nchain
/home/gcasale/Dropbox/code/line-dev.git/python/line_solver/api/solvers/mva/amvald.py:119: RuntimeWarning: invalid value encountered in divide
  deltaclass = (Nchain_in - 1) / Nchain_in


Station    JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay ClosedClass 1.3844 1.3844 0.6655  0.6655 2.0778 2.0778
 Queue1 ClosedClass 0.6156 0.3958 0.2971  0.2971 2.0778 2.0778
 Queue1   OpenClass 0.1207 0.0998 1.2465  1.2465 0.0998 0.0998
 Source ClosedClass 0.0000 0.0000 0.0000  0.0000 0.0000 1.0000
 Source   OpenClass 0.0000 0.0000 0.0000  0.0000 0.0000 0.1000
  Station     JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay  ClosedClass  1.3844  1.3844  0.6655  0.6655  2.0778  2.0778
1  Queue1  ClosedClass  0.6156  0.3958  0.2971  0.2971  2.0778  2.0778
2  Queue1    OpenClass  0.1207  0.0998  1.2465  1.2465  0.0998  0.0998
3  Source  ClosedClass  0.0000  0.0000  0.0000  0.0000  0.0000  1.0000
4  Source    OpenClass  0.0000  0.0000  0.0000  0.0000  0.0000  0.1000
